# Homography transform practice 

Homography - mathematical transformation that maps points in one plane to points in another plane

Goal: accomplish this with ground plane points for a COLMAP cloud

Create a top-down cloud of the SfM generated cloud 

To be used for satellite matching 

FIRST: Transform COLMAP coordinates to a reference image frame of my choosing 

CHECKLIST

- Get pose 1 with known points on the ground plane
- Get points from that recognized pose 
- Choose a reference camera location
- Get a translation and rotation for that camera location
- Perform matrix math on each point in COLMAP --> translate to ref coordinates
- Plot asterisk to confirm same location

In [1]:
import numpy as np
import cv2
from skimage import *
from skimage import feature
import math
import matplotlib.pyplot as plt
import copy

from colmapParsingUtils import *
from ptCloudParsingUtils import *


from scipy.spatial.distance import euclidean
from skimage import io
from skimage import color
from skimage.transform import resize
import math
from skimage.feature import hog

%load_ext autoreload
%autoreload 2
%autosave 180

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


### Load point cloud data 

In [2]:
# # Scene: Tufts Turf Scene (generated by COLMAP with personal photos)
# imagedata_fn = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/images.txt"
# cameradata_fn = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/cameras.txt"
# pointdata_fn = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/points3D.txt"
# images_colmap = read_images_text('/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/images.txt')
# cameras = read_cameras_text('/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/cameras.txt')
# pts3d = read_points3D_text('/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/points3D.txt')

# Scene: Tufts Walkway Scene (generated by COLMAP with personal photos)
imagedata_fn = "/home/daniel-choate/ASAR/s2/TerrainNav/TuftsWalkway/images.txt"
cameradata_fn = "/home/daniel-choate/ASAR/s2/TerrainNav/TuftsWalkway/cameras.txt"
pointdata_fn = "/home/daniel-choate/ASAR/s2/TerrainNav/TuftsWalkway/points3D.txt"
images_colmap = read_images_text('/home/daniel-choate/ASAR/s2/TerrainNav/TuftsWalkway/images.txt')
cameras = read_cameras_text('/home/daniel-choate/ASAR/s2/TerrainNav/TuftsWalkway/cameras.txt')
pts3d = read_points3D_text('/home/daniel-choate/ASAR/s2/TerrainNav/TuftsWalkway/points3D.txt')

In [3]:
# print(images_colmap)
# print(cameras)
# print(pts3d)

pts3D
- point id number
- xyz position (in relation to colmap coordinates
- rgb value
- Reprojection error
- image_ids: Which images the point was observed in
- point2D_idxs: The 2D feature matches (same length as image_ids)
    - An index into the list of detected 2D keypoints in a specific image
    - Features like corners, edges, or interested points

In [4]:
# Extract 3d points from raw file 

# pointdata_fn = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/points3D.txt"
pointdata_fn = "/home/daniel-choate/ASAR/s2/TerrainNav/TuftsWalkway/points3D.txt"

num_pts = 75497 # Gerrard Hall

# points_data = []
scene_pts = np.zeros([num_pts, 3]) 
idx = 0

with open(pointdata_fn, 'r') as file:
    lines = file.readlines()[3:]  # Skip the first three lines
    
    for line in lines:
        elements = line.strip().split()
        
        row = [float(elem) for elem in list(elements)]

        scene_pts[idx,0] = row[1]
        scene_pts[idx,1] = row[2]
        scene_pts[idx,2] = row[3]
        idx += 1

# Filter RGB data into separate numpy array 
rgb_data = np.zeros([num_pts, 3])
idx = 0

with open(pointdata_fn, 'r') as file:
    lines = file.readlines()[3:] # Skip the first three lines 
    for line in lines:
        elements = line.strip().split()
        row = [float(elem) for elem in list(elements)]

        rgb_data[idx,0] = row[4]
        rgb_data[idx,1] = row[5]
        rgb_data[idx,2] = row[6]
        idx += 1

# SCALE COLORS
rgb_data = rgb_data/255
# print(scene_pts[:10])

# Filter RGB data into separate numpy array with ALPHA
rgba_data = np.zeros([num_pts, 4])
idx = 0

with open(pointdata_fn, 'r') as file:
    lines = file.readlines()[3:] # Skip the first three lines 
    for line in lines:
        elements = line.strip().split()
        row = [float(elem) for elem in list(elements)]

        rgba_data[idx,0] = row[4]
        rgba_data[idx,1] = row[5]
        rgba_data[idx,2] = row[6]
        rgba_data[idx,3] = .5
        idx += 1

# SCALE COLORS
# rgba_data = rgba_data/255
rgba_data[:, :3] = rgba_data[:, :3] / 255
# print(scene_pts[:10])

# Get camera poses 

In [5]:
# Convert COLMAP poses (xyz,quats) to rotm

# Ceate an empty array for camera poses as 4x4 transformation matrixes
poses = np.zeros([len(images_colmap),4,4])
# Create an empty array for image data, for 250x250 pixels 
images = np.zeros([len(poses),250,250,3])


skip_indices = {10, 17, 65, 83, 84}  # Use a set to store the indices to skip

#loop through <images_from_colmap> to get 3D poses of cameras at each timestamp
# print(len(images_colmap))
for n in range(len(images_colmap)):
    # print(n)
    if n in skip_indices:
        continue


    # Pull quaternion and translation vector
    qvec = images_colmap[n+1].qvec #raw
    # print(qvec)
    # print(images_colmap[n+1].tvec)
    tvec = images_colmap[n+1].tvec[:,None]
    # print(tvec)
    
    t = tvec.reshape([3,1])
    # print(tvec)
    Rotmat = qvec2rotmat(-qvec)
    
    bottom = np.array([0.0, 0.0, 0.0, 1.0]).reshape([1, 4])
    # m represents world-to-camera transformation 
    # c2w represents camera-to-world transformation 
    m = np.concatenate([np.concatenate([Rotmat, t], 1), bottom], 0)
    c2w = np.linalg.inv(m)

    # NERF SPECIFIC ROTATIONS 
    # c2w[0:3,2] *= -1 # flip the y and z axis
    # c2w[0:3,1] *= -1
    # c2w = c2w[[1,0,2,3],:]
    # c2w[2,:] *= -1 # flip whole world upside down
    


    poses[n] = c2w
    # if n == 5:
    #     print(c2w)
        
    # poses[n] = m
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#GET REST OF PARAMS NEEDED FOR tinyNeRF format~~~~~~~~~~~~~~~~~~~~~~~~~~~~    

#fix order of colors
images[:,:,:,0], images[:,:,:,1] = images[:,:,:,1], images[:,:,:,0]

H,W = images.shape[1:3]
# print(H,W)
testimg, testpose = images[55], poses[55]

focal = cameras[1].params[0] #test- see if same focal length can be shared across all images
focal1 = focal/12
print(focal)
# print(poses)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# print(poses[5])

3011.9738700121015


## CALCULATE FOCAL LENGTH 

- f_mm = focal length in mm
- f_pix = focal length in pixel 
- w_sensor = physical width of the camera sensor in millimeters (camera specs)
- w_image = width of the image in pixels (horizontal resolution)

f_mm = (f_pix x w_sensor) / w_im

# Plotting point cloud with origin

In [6]:
# Plotting 3D points
# BY ITSELF
# Data generated by COLMAP

vis = o3d.visualization.Visualizer()
vis.create_window()

# Add coordinate axes
axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)


scene_cloud = o3d.geometry.PointCloud()
scene_cloud.points = o3d.utility.Vector3dVector(scene_pts)
scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgba_data)

# Add necessary geometries
vis.add_geometry(axis_origin)
vis.add_geometry(scene_cloud)

render_option = vis.get_render_option()
render_option.point_size = 2

# Run visualization 
vis.run()
vis.destroy_window()


libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 69
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 69
pci id for fd 69: 8086:a7a0, driver iris
MESA-LOADER: dlopen(/usr/lib/x86_64-linux-gnu/dri/iris_dri.so)
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using 

# Plotting point cloud with camera poses 

In [7]:
# # Create Open3D visualizer object
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name='3D Plot with Pose Axes and Point Cloud', width=1000, height=1000)

# # Add coordinate axes
# axis_orig = o3d.geometry.TriangleMesh.create_coordinate_frame(size=.25)
# axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)

# # loop through poses 
# for pose in poses:
#     homog_t = pose
#     axes1 = copy.deepcopy(axes).transform(homog_t)
#     vis.add_geometry(axes1)
    

# vis.add_geometry(axis_orig)
# vis.add_geometry(scene_cloud)

# render_option = vis.get_render_option()
# render_option.point_size = 2

# # Run the visualizer
# vis.run()
# vis.destroy_window()

# Choosing a specific image 

In [8]:
# Choose an image id
image_id = 40
# Gather 3d point ids from image data 
im_Ids_o = images_colmap[image_id].point3D_ids
# print(images_colmap[image_id])
# print(im_Ids_o)
# print(im_Ids_o[1])
# Loop through and remove null 3d Ids
im_Ids = []
for i in range(len(im_Ids_o)):
    if im_Ids_o[i] > 0:
        im_Ids.append(im_Ids_o[i])

# print(im_Ids)
# Gather xyz points for each 3d point ID
camera_pts_xyz = np.zeros((len(im_Ids),3))
camera_pts_rgb = np.zeros((len(im_Ids),3))
# print(len(camera_pts_xyz))
# print(len(im_Ids))
for i in range(len(im_Ids)):
    camera_pts_xyz[i] = pts3d[im_Ids[i]].xyz
    camera_pts_rgb[i] = pts3d[im_Ids[i]].rgb
    # print(pts3d[im_Ids[i]].rgb)

# Get camera pose for chosen image 
def get_pose_id(id):
    qvec = images_colmap[id].qvec
    tvec = images_colmap[id].tvec[:,None]
    # print(tvec)
    
    t = tvec.reshape([3,1])
    # print(tvec)
    Rotmat = qvec2rotmat(-qvec)
    
    bottom = np.array([0.0, 0.0, 0.0, 1.0]).reshape([1, 4])
    m = np.concatenate([np.concatenate([Rotmat, t], 1), bottom], 0)
    c2w = np.linalg.inv(m)
    return c2w

# Normalize colors 
camera_pts_rgb = camera_pts_rgb/255

homog_t = get_pose_id(image_id)
# print(homog_t)
rotation_m = homog_t[:3,:3]
# print(rotation_m)
# Gather translation vector 
trans_v = homog_t[:3,-1]
# print(trans_v)
size = 1

# print(camera_pts_xyz)
# print(pts3d[im_Ids[0]].xyz)
# print(pts3d[26358].xyz)
# print(len(images_colmap))
# print(len(images_colmap[1].point3D_ids))

In [9]:
# # Plotting points with regular point cloud 

# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name='3D points from specified image id', width=1000, height=1000)

# # Create initial axes 
# axis_orig = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=np.array([0.,0.,0]))
# axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.25, origin = np.array([0.,0.,0.])).transform(homog_t)

# # axes = axes.transform(homog_t)

# camera_specific = o3d.geometry.PointCloud()
# camera_specific.points = o3d.utility.Vector3dVector(camera_pts_xyz)
# camera_specific.paint_uniform_color([0, 0, 1])



# vis.add_geometry(axis_orig)
# vis.add_geometry(axes)
# vis.add_geometry(scene_cloud)
# vis.add_geometry(camera_specific)


# # render_option = vis.get_render_option()

# # render_option.background_color = [1, 1, 1] #[0.678, 0.847, 0.902] # Light blue

# vis.run()

# vis.destroy_window()


# Transform colmap coordinates to relative camera coordinates 

Inverse of a Homogeneous Transformation Matrix

A homogeneous transformation matrix T = \begin{bmatrix}
R & t \\
0 & 1
\end{bmatrix}

Steps to Compute \( T^{-1} \)
1. R^{-1} = R^T

2. Multiply negative of the translation vector \( t \) by \( R^T \):  
   t' = -R^T t

3. Form the new homogeneous transformation matrix: 
   Combine the results into a new homogeneous transformation matrix:
   \[
   T^{-1} = \begin{bmatrix}
   R^T & -R^T t \\
   0 & 1
   \end{bmatrix}


Example
If T = \begin{bmatrix}
\cos\theta & -\sin\theta & tx \\
\sin\theta & \cos\theta & ty \\
0 & 0 & 1
\end{bmatrix}

then T^{-1} = \begin{bmatrix}
\cos\theta & \sin\theta & -tx\cos\theta - ty\sin\theta \\
-\sin\theta & \cos\theta & tx\sin\theta - ty\cos\theta \\
0 & 0 & 1
\end{bmatrix}.

In [10]:
def inv_homog_transform(homog):
    """ Inverting a homogeneous transformation matrix
    Inputs: homogeneous transformation matrix (4x4)
    Outputs: inverted 4x4 matrix
    """
    # Grab rotation matrix
    R = homog[:3,:3]
    print("\n R \n",R)
    # Transpose rotation matrix 
    R_inv = R.T
    print("\n Transpose of R \n",R_inv)
    # Grab translation matrix 
    t = homog[:-1, -1]
    t = t.reshape((3, 1))
    print("\n T \n",t)
    # - R_t * t
    t_inv = -R_inv @ t
    print("\n -R_t * T \n",t_inv)
    bottom = np.array([0.0, 0.0, 0.0, 1.0]).reshape([1, 4])
    homog_inv = np.concatenate([np.concatenate([R_inv, t_inv], 1), bottom], 0)    
    print("\n Homogeneous new = \n", homog_inv)
    return homog_inv

print("Homogeneous transformation \n", homog_t)
homog_inv = inv_homog_transform(homog_t)

Homogeneous transformation 
 [[ 9.95185130e-01 -9.79585997e-02 -3.26632264e-03  9.55132609e-01]
 [ 9.73658918e-02  9.91888367e-01 -8.17150572e-02  2.92746441e-01]
 [ 1.12445200e-02  8.10035814e-02  9.96650380e-01 -3.76046631e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

 R 
 [[ 0.99518513 -0.0979586  -0.00326632]
 [ 0.09736589  0.99188837 -0.08171506]
 [ 0.01124452  0.08100358  0.99665038]]

 Transpose of R 
 [[ 0.99518513  0.09736589  0.01124452]
 [-0.0979586   0.99188837  0.08100358]
 [-0.00326632 -0.08171506  0.99665038]]

 T 
 [[ 0.95513261]
 [ 0.29274644]
 [-3.76046631]]

 -R_t * T 
 [[-0.93675265]
 [ 0.1078029 ]
 [ 3.77491174]]

 Homogeneous new = 
 [[ 9.95185130e-01  9.73658918e-02  1.12445200e-02 -9.36752650e-01]
 [-9.79585997e-02  9.91888367e-01  8.10035814e-02  1.07802903e-01]
 [-3.26632264e-03 -8.17150572e-02  9.96650380e-01  3.77491174e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [11]:
# Re-establish necessary variables 
image_id = image_id
homog_t = get_pose_id(image_id)
print("Homog \n", homog_t)
print("\n Inverse \n", homog_inv)

# Inverse of homogeneous transformation to 

pts_transformed = np.zeros((len(camera_pts_xyz),3))

# Apply homogeneous transform for each point in cloud
print(camera_pts_xyz)

for i in range(len(camera_pts_xyz)):
    pt = camera_pts_xyz[i]
    # Change point from 3x1 to 4x1 for homogeneous transform 
    pt = np.append(pt, 1)
    print(pt)
    new_pt = pt @ homog_inv
    print(new_pt)
    # Grab xyz coodinates of new point
    pt_t = new_pt[:3]
    print(pt_t)
    pts_transformed[i] = pt_t


Homog 
 [[ 9.95185130e-01 -9.79585997e-02 -3.26632264e-03  9.55132609e-01]
 [ 9.73658918e-02  9.91888367e-01 -8.17150572e-02  2.92746441e-01]
 [ 1.12445200e-02  8.10035814e-02  9.96650380e-01 -3.76046631e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

 Inverse 
 [[ 9.95185130e-01  9.73658918e-02  1.12445200e-02 -9.36752650e-01]
 [-9.79585997e-02  9.91888367e-01  8.10035814e-02  1.07802903e-01]
 [-3.26632264e-03 -8.17150572e-02  9.96650380e-01  3.77491174e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 0.22837523 -0.48802787 -2.43451632]
 [ 0.18567003 -0.51637226 -2.33835711]
 [ 0.24137813 -0.41947745 -2.45707688]
 ...
 [-1.62793234 -0.09502431  0.51277903]
 [ 0.87674574  0.09341337 -3.05017452]
 [ 3.06875232 -2.73556686  7.55315384]]
[ 0.22837523 -0.48802787 -2.43451632  1.        ]
[ 0.28303408 -0.26289657 -2.46332565 -8.45662616]
[ 0.28303408 -0.26289657 -2.46332565]
[ 0.18567003 -0.51637226 -2.33835711  1.        ]
[ 0.24299698 -

In [12]:
# PLOT OLD POINTS

vis = o3d.visualization.Visualizer()
vis.create_window()

# Add coordinate axes
axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)
# axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.25, origin = np.array([0.,0.,0.])).transform(homog_t)
axes1 = copy.deepcopy(axes).transform(homog_inv)



pts_old = o3d.geometry.PointCloud()
pts_old.points = o3d.utility.Vector3dVector(camera_pts_xyz)
pts_old.paint_uniform_color([0, 0, 1])
# pts_old.colors = o3d.utility.Vector3dVector(camera_pts_rgb)

pts_new = o3d.geometry.PointCloud()
pts_new.points = o3d.utility.Vector3dVector(pts_transformed)
# pts_new.paint_uniform_color([1, 0, 0])
pts_new.colors = o3d.utility.Vector3dVector(camera_pts_rgb)

# Add necessary geometries
# vis.add_geometry(axis_origin)
vis.add_geometry(axes)
vis.add_geometry(axes1)
vis.add_geometry(scene_cloud)
vis.add_geometry(pts_old)
vis.add_geometry(pts_new)


render_option = vis.get_render_option()
render_option.point_size = 2

# Run visualization 
vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 77
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 77
pci id for fd 77: 8086:a7a0, driver iris
MESA-LOADER: dlopen(/usr/lib/x86_64-linux-gnu/dri/iris_dri.so)
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using 

In [13]:
# # PLOT NEW POINTS 

# vis = o3d.visualization.Visualizer()
# vis.create_window()

# # Add coordinate axes
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)
# axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)


# pts_new = o3d.geometry.PointCloud()
# pts_new.points = o3d.utility.Vector3dVector(pts_transformed)
# pts_new.paint_uniform_color([0, 0, 1])
# # pts_old.colors = o3d.utility.Vector3dVector(rgb_data)

# # Add necessary geometries
# vis.add_geometry(axis_origin)
# vis.add_geometry(pts_new)

# render_option = vis.get_render_option()
# render_option.point_size = 2

# # Run visualization 
# vis.run()
# vis.destroy_window()

# Transform from 3d to 2d points

In [14]:
# Create new 2D point array

pts_2d = np.zeros((len(pts_transformed),3))

for i in range(len(pts_transformed)):
    pt = pts_transformed[i,:]
    # print(pt)
    x = pt[0]
    y = pt[1]
    z = pt[2]
    x_new = x/z
    # print(x_new)
    y_new = y/z
    # print(y_new)
    new_pt = np.array((x_new, y_new, 1))
    print(new_pt)
    pts_2d[i,:] = new_pt

print(pts_2d)

[-0.11489917  0.10672425  1.        ]
[-0.10251892  0.1278451   1.        ]
[-0.11666127  0.07733226  1.        ]
[-0.27770434 -0.12938352  1.        ]
[-0.30460142 -0.14092586  1.        ]
[-0.27693745 -0.12852068  1.        ]
[-0.2887646  -0.13419091  1.        ]
[-0.30814118 -0.14281114  1.        ]
[-0.298588   -0.13953912  1.        ]
[-0.28969464 -0.13482412  1.        ]
[-0.28969464 -0.13482412  1.        ]
[-0.28090896 -0.13145467  1.        ]
[-0.28095785 -0.13135685  1.        ]
[-0.29009213 -0.1339674   1.        ]
[-0.30354981 -0.14031789  1.        ]
[-0.29867055 -0.13855338  1.        ]
[-0.29442499 -0.13666916  1.        ]
[-0.2935878  -0.13636041  1.        ]
[-0.30478439 -0.14029553  1.        ]
[-0.27699336 -0.11929053  1.        ]
[-0.0874203   0.08881202  1.        ]
[-0.08604466  0.09119453  1.        ]
[-0.2794865  -0.12840154  1.        ]
[-0.29793024 -0.13714248  1.        ]
[-0.29926652 -0.13842056  1.        ]
[-0.27626689 -0.11705744  1.        ]
[-0.28222276

In [15]:
# PLOT NEW POINTS 

vis = o3d.visualization.Visualizer()
vis.create_window()

# Add coordinate axes
axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=.5)
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)


cloud_2d = o3d.geometry.PointCloud()
cloud_2d.points = o3d.utility.Vector3dVector(pts_2d)
# cloud_2d.paint_uniform_color([0, 0, 1])
cloud_2d.colors = o3d.utility.Vector3dVector(camera_pts_rgb)

# Add necessary geometries
vis.add_geometry(axis_origin)
vis.add_geometry(cloud_2d)

render_option = vis.get_render_option()
render_option.point_size = 2

# Run visualization 
vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 76
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 76
pci id for fd 76: 8086:a7a0, driver iris
MESA-LOADER: dlopen(/usr/lib/x86_64-linux-gnu/dri/iris_dri.so)
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using 